In [13]:
import selenium
from selenium import webdriver
import time
import pandas as pd

In [14]:
def get_drive_launch(url):
    driver = webdriver.Chrome('./driver/chromedriver.exe')
    driver.maximize_window()
    driver.implicitly_wait(10)
    driver.get(url)
    return driver


In [15]:
def get_highlevel_details_naukri(job, location = None, salary_range=None, stop=None, use_filter=False):
    url = 'https://www.naukri.com/'
    drivr = get_drive_launch(url)

    drivr.find_element_by_id("qsb-keyword-sugg").send_keys(job)
    if ((location) and (not use_filter)):
        drivr.find_element_by_id('qsb-location-sugg').send_keys(location)
    time.sleep(2)
    drivr.find_element_by_xpath("//div[@class='search-btn']/button").click()
    #click to expand location filter
    
    
    if use_filter:        
        if location:
            time.sleep(3)
            
            try:
                drivr.find_element_by_xpath("//span[@class = 'ellipsis fleft' and contains(text(), "+"'"+location+"'"+")]/../preceding-sibling::i").click()
            except:
                drivr.find_element_by_xpath("//div[@data-filter-id='citiesGid']//a").click()
                time.sleep(3)
                drivr.find_element_by_xpath("//span[@class = 'ellipsis fleft' and contains(text(), "+"'"+location+"'"+")]/../preceding-sibling::i").click()
        if salary_range:

            try:
                drivr.find_element_by_xpath("//span[contains(text(), "+"'"+salary_range+"'"+")]/../preceding-sibling::i").click()
            except:
                drivr.find_element_by_xpath("//div[@data-filter-id='salaryRange']//a").click()
                time.sleep(3)
                drivr.find_element_by_xpath("//span[contains(text(), "+"'"+salary_range+"'"+")]/../preceding-sibling::i").click()

    time.sleep(5)
    print(drivr.find_element_by_xpath("//div[@class='sortAndH1Cont']/h1").text)
    titles=[]
    companies=[]
    exps=[]
    salaries=[]
    locs=[]

    job_titles = drivr.find_elements_by_xpath("//div[@class='info fleft']/a")
    job_companies = drivr.find_elements_by_xpath("//div[@class='info fleft']/div/a[1]")
    job_exps = drivr.find_elements_by_xpath("//div[@class='info fleft']/ul/li[1]")
    job_salaries = drivr.find_elements_by_xpath("//div[@class='info fleft']/ul/li[2]")
    job_locations = drivr.find_elements_by_xpath("//div[@class='info fleft']/ul/li[3]")
    for job_title, job_company, job_exp, job_salary, job_loc in zip(job_titles[:stop],job_companies[:stop],job_exps[:stop],job_salaries[:stop],job_locations[:stop]):
        titles.append(job_title.text)
        companies.append(job_company.text)
        exps.append(job_exp.text)
        salaries.append(job_salary.text)
        locs.append(job_loc.text)
    
    return drivr, pd.DataFrame({'Job Title':titles, 'Company':companies, 'Experience Required':exps,'Salary offered':salaries, 'Location':locs})

In [16]:
def get_full_descrp_naukri(driver, stop = None):
    main_window = driver.current_window_handle
    all_windows = driver.window_handles

    for window in all_windows:
        if window != main_window:
            driver.switch_to.window(window)
            driver.close()

    driver.switch_to.window(main_window)
    main_window = driver.current_window_handle
    all_windows = driver.window_handles

    full_descriptions = []

    for element in driver.find_elements_by_xpath("//div[@class='info fleft']/a")[:stop]:
        element.click()
        time.sleep(2)
        all_windows = driver.window_handles
        driver.switch_to.window(all_windows[1])
        try:
            full_descriptions.append(driver.find_element_by_xpath("//section[@class='job-desc']").text)
        except:
            full_descriptions.append(driver.find_element_by_xpath("//div[@class='clearboth description']").text)
        driver.close()
        driver.switch_to.window(main_window)
    return driver, full_descriptions


In [17]:
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException

In [18]:
def login_glassdoor(driver):
    username = 'testacount573@gmail.com'
    paswd = 'Testaccount123'
    
    main_window = driver.current_window_handle
    try:
        driver.find_element_by_class_name("locked-home-sign-in").click()
    except NoSuchElementException:
        driver.find_element_by_class_xpath("//div[@class='d-flex']//button").click()
        
    time.sleep(3)
    #all_windows = driver.window_handles
    #print(all_windows)
    #driver.switch_to.window(all_windows[1])
    
    driver.find_element_by_id("userEmail").send_keys(username)
    driver.find_element_by_id("userPassword").send_keys(paswd)
    driver.find_element_by_xpath("//button[contains(text(),'Sign In')]").click()
    time.sleep(3)
    driver.switch_to.window(main_window)
    return driver

In [19]:
def get_highlevel_details_glassdoor(job, location = None, salary_range=None, stop=None, use_filter=False):
    url = 'https://www.glassdoor.co.in/index.htm'
    drivr = get_drive_launch(url)
    drivr = login_glassdoor(drivr)
    drivr.find_element_by_id("sc.keyword").send_keys(job)
    time.sleep(5)
    
    if ((location) and (not use_filter)):
        drivr.find_element_by_xpath("//input[@data-test='search-bar-location-input']").send_keys(Keys.CONTROL + "a")
        drivr.find_element_by_xpath("//input[@data-test='search-bar-location-input']").send_keys(Keys.DELETE)
        drivr.find_element_by_xpath("//input[@data-test='search-bar-location-input']").send_keys(location)
    time.sleep(2)
    drivr.find_element_by_xpath("//button[@type='submit']").click()
    #click to expand location filter 
    if use_filter:        
        if location:
            time.sleep(3)
            
            try:
                drivr.find_element_by_xpath("//span[@class = 'ellipsis fleft' and contains(text(), "+"'"+location+"'"+")]/../preceding-sibling::i").click()
            except:
                drivr.find_element_by_xpath("//div[@data-filter-id='citiesGid']//a").click()
                time.sleep(3)
                drivr.find_element_by_xpath("//span[@class = 'ellipsis fleft' and contains(text(), "+"'"+location+"'"+")]/../preceding-sibling::i").click()
        if salary_range:

            try:
                drivr.find_element_by_xpath("//span[contains(text(), "+"'"+salary_range+"'"+")]/../preceding-sibling::i").click()
            except:
                drivr.find_element_by_xpath("//div[@data-filter-id='salaryRange']//a").click()
                time.sleep(3)
                drivr.find_element_by_xpath("//span[contains(text(), "+"'"+salary_range+"'"+")]/../preceding-sibling::i").click()

    time.sleep(5)
    print(drivr.find_element_by_xpath("//h1[@data-test='jobTitle']").text)
    titles=[]
    companies=[]
    no_days_posts=[]
    #salaries=[]
    locs=[]
    ratings=[]
    
    job_links = drivr.find_elements_by_xpath("//div[@class='d-flex flex-column css-x75kgh e1rrn5ka3']")
    
    for job_link in job_links[:stop]:
        try:
            if job_link.text:
                ratings.append(job_link.text)
            else:
                ratings.append('-')
            #ratings.append(job_link.find_element_by_xpath("//span[@class='css-19pjha7 e1cjmv6j1']").text)
        except:
            ratings.append("No Rating")
    job_titles = drivr.find_elements_by_xpath("//a[@data-test='job-link']/span")
    job_companies = drivr.find_elements_by_xpath("//div[@class='d-flex justify-content-between align-items-start']//a/span")
    job_days = drivr.find_elements_by_xpath("//div[@data-test='job-age']")
    #job_salaries = drivr.find_elements_by_xpath("//div[@class='info fleft']/ul/li[2]")
    job_locations = drivr.find_elements_by_xpath("//div[@class='d-flex flex-wrap css-11d3uq0 e1rrn5ka2']/span")
    
    for job_title, job_company, job_day, job_loc in zip(job_titles[:stop],job_companies[:stop],job_days[:stop],job_locations[:stop]):
        titles.append(job_title.text.strip())
        companies.append(job_company.text.strip())
        no_days_posts.append(job_day.text.strip())
        #salaries.append(job_salary.text)
        locs.append(job_loc.text.strip())
    
    return drivr, pd.DataFrame({'Job Title':titles, 'Company':companies, 'Job Post Age':no_days_posts, 'Location':locs, 'Ratings':ratings})

In [20]:
def get_product_highlevel_details_flipkart(product_name, no_of_products = 100):
    url = 'https://www.flipkart.com/'
    driver = get_drive_launch(url)
    driver.find_element_by_xpath("//button[contains(text(),'✕')]").click()

    driver.find_element_by_xpath("//input[@title='Search for products, brands and more']").send_keys(product_name)
    driver.find_element_by_xpath("//button[@type='submit']").click()
    time.sleep(2)

    product_names = []
    product_descriptions = []
    product_prices = []
    product_discounts = []

    while len(product_names) < no_of_products:
        product_name_elems = driver.find_elements_by_xpath("//div[@class='_2WkVRV']")
        product_description_elems = driver.find_elements_by_xpath("//div[@class='_2WkVRV']/../a[not(@class='_3bPFwb')]")
        price_elems= driver.find_elements_by_xpath("//a[@class='_3bPFwb']//div[@class='_30jeq3']")
        price_discount_elems = driver.find_elements_by_xpath("//a[@class='_3bPFwb']")
        for prod_name_elem, product_description_elem, price_elem, price_discount_elem in zip(product_name_elems,product_description_elems,price_elems,price_discount_elems):
            product_names.append(prod_name_elem.text.strip())
            product_descriptions.append(product_description_elem.text.strip())
            product_prices.append(price_elem.text.strip())
            try:
                discount = price_discount_elem.find_element_by_class_name("_3Ay6Sb").text
            except:
                discount = None
            if discount:
                product_discounts.append(discount)
            else:
                product_discounts.append("0% off")

            if len(product_names) == no_of_products:
                break
        driver.find_element_by_xpath("//span[contains(text(),'Next')]").click()
        time.sleep(3)
    dataframe = pd.DataFrame({'Product Brand': product_names, 'Product Description' : product_descriptions, 'Product Price' : product_prices,'Product Discount':product_discounts})
    return driver, dataframe

## Q1: Write a python program to scrape data for “Data Analyst” Job position in “Bangalore” location. You have to scrape the job-title, job-location, company_name, experience_required. You have to scrape first 10 jobs data.


This task will be done in following steps:
1. first get the webpage https://www.naukri.com/
2. Enter “Data Analyst” in “Skill,Designations,Companies” field and enter “Bangalore” in “enter the location” field.
3. Then click the search button.
4. Then scrape the data for the first 10 jobs results you get.
5. Finally create a dataframe of the scraped data. 

Note- All of the above steps have to be done in code. No step is to be done manually

In [64]:
d, dataFrame = get_highlevel_details_naukri('Data Analyst', location='Bangalore', stop=10)
d.quit()
dataFrame

Data Analyst Jobs In Bangalore


,Job Title,Company,Experience Required,Salary offered,Location
0,Assistant Manager II - Data Analyst,Flipkart Internet Private Limited,4-8 Yrs,Not disclosed,Bangalore/Bengaluru
1,Data Analyst,Super India Tech Mark,0-2 Yrs,"1,25,000 - 2,25,000 PA.",Bangalore/Bengaluru(Devalapur)
2,Data Analyst,tech mahindra ltd,4-8 Yrs,Not disclosed,Bangalore/Bengaluru
3,Data Analyst,GlaxoSmithKline Pharmaceuticals Limited,2-7 Yrs,Not disclosed,Bangalore/Bengaluru
4,Data Analyst,Xiaomi,2-6 Yrs,Not disclosed,Bangalore/Bengaluru
5,Hiring For Data Analyst,Concentrix Daksh Services India Private Limited.,2-7 Yrs,"4,00,000 - 9,00,000 PA.","Hyderabad/Secunderabad, Pune, Chennai, Bangalo..."
6,Senior Data Analyst,Cerner,3-5 Yrs,Not disclosed,Bangalore/Bengaluru
7,Data Analyst with Marketing Analytics-Capco,Capco Technologies Pvt Ltd,4-9 Yrs,"7,00,000 - 17,00,000 PA.","Pune, Bangalore/Bengaluru"
8,Business Data Analyst - MIS & Reporting,INTERTRUST GROUP,3-7 Yrs,Not disclosed,"Mumbai, Bangalore/Bengaluru"
9,Junior Data Analyst,Happy Marketer Private Ltd,1-3 Yrs,"3,00,000 - 7,00,000 PA.","Bangalore/Bengaluru, Mumbai (All Areas)"


## Q2: Write a python program to scrape data for “Data Scientist” Job position in “Bangalore” location. You have to scrape the job-title, job-location, company_name, full job-description. You have to scrape first 10 jobs data.


This task will be done in following steps:
1. first get the webpage https://www.naukri.com/
2. Enter “Data Scientist” in “Skill,Designations,Companies” field and enter “Bangalore” in “enter the location” field.
3. Then click the search button.
4. Then scrape the data for the first 10 jobs results you get.
5. Finally create a dataframe of the scraped data.

Note- 
1. All of the above steps have to be done in code. No step is to be done manually.
2. Please note that you have to scrape full job description. For that you may have to open each job separately as shown below

In [65]:
d, dataFrame = get_highlevel_details_naukri('Data Scientist', location='Bangalore', stop=10)
d, dataFrame['Full Description'] = get_full_descrp_naukri(d, 10)
d.quit()
dataFrame.drop(['Experience Required', 'Salary offered'], axis=1, inplace = True)
dataFrame

Data Scientist Jobs In Bangalore


,Job Title,Company,Location,Full Description
0,Required- Data Scientist (NLP)-Axis Bank - 6 m...,Axis Bank Limited,"Kolkata, New Delhi, Hyderabad/Secunderabad, Pu...",Roles and Responsibilities - The bank generate...
1,Data Scientist - Python/ MATLAB/ Machine Learn...,Wrackle Technologies Pvt Ltd,Bangalore/Bengaluru,Job description\nData Scientist - Data Mining/...
2,Principal Data Scientist - Machine/Deep Learni...,Fidius advisory,Bangalore/Bengaluru,Job description\nJob Description :\n- We are l...
3,Data Scientist/ Analyst,Becton Dickinson India Pvt. Ltd,Bangalore/Bengaluru,Job description\nRoles and Responsibilities\no...
4,Opportunity For Data Scientist Internship - Be...,Corner Stone Solutions,Bangalore/Bengaluru,Job description\nLocation - Bangalore / Bengal...
5,Data Scientist || Data Analyst || Data science,Inspiration Manpower Consultancy Pvt. Ltd.,"Navi Mumbai, Bangalore/Bengaluru",Job description\nJob description\nJob Summary ...
6,"Senior Data Scientist, , Data Science",Visa Inc.,Bangalore/Bengaluru,Job description\nWe are seeking an innovative ...
7,Lead Data Scientist - Immediate Joiners are Re...,Techolution India Private Limited,"Mumbai, Hyderabad/Secunderabad, Pune, Chennai,...",Job description\nWe are looking for qualified ...
8,Senior Data Scientist (Machine Vision solutions),ONX Software Systems Pvt Ltd,Bangalore/Bengaluru,Job description\nSenior Data Scientist (Machin...
9,Global Medical Data Scientist,GlaxoSmithKline Pharmaceuticals Limited,Bangalore/Bengaluru,This is an ideal role for an experienced candi...


In [66]:
dataFrame.loc[2, 'Full Description']

"Job description\nJob Description :\n- We are looking for a researcher who specializes in building personalization/recommender systems algorithm (ML APIs) for the applications mentioned above and work with our engineers to deploy them as scale.\nWhat you will do :\n- Apply your research expertise to build our ML-driven recommender system products, help us develop new solutions and unlock new directions, as well as analyse and optimise the systems we already. \n- You'll work closely with product teams and mentor them on best practices for modern ML, and keep the wider team informed on the state-of-the-art. In addition, you will be in a strategic position to influence future roadmaps for recommender system products.\n- Collaborate with a cross-functional agile team spanning user research, design, data science, product management, and engineering to build new product features that advance our mission to connect artists and fans in personalized and relevant ways.\n- Prototype new approache

## Q3: In this question you have to scrape data using the filters available on the webpage as shown below:


1. You have to use the location and salary filter.
2. You have to scrape data for “Data Scientist” designation for first 10 job results.
3. You have to scrape the job-title, job-location, company_name, experience_required.

1. The location filter to be used is “Delhi/NCR”
2. The salary filter to be used is “3-6” lakhs

The task will be done as shown in the below steps:
1. first get the webpage https://www.naukri.com/
2. Enter “Data Scientist” in “Skill,Designations,Companies” field .WEB SCRAPING ASSIGNMENT-2.
3. Then click the search button.
4. Then apply the location filter and salary filter by checking the respective boxes
5. Then scrape the data for the first 10 jobs results you get.
6. Finally create a dataframe of the scraped data.

Note- All of the above steps have to be done in code. No step is to be done 
manually.

In [67]:
driver,dataFrame=get_highlevel_details_naukri('Data Scientist', location = 'Delhi / NCR', salary_range='3-6 Lakhs', stop=10, use_filter=True)
driver.quit()
dataFrame.drop(['Salary offered'],axis=1,inplace=True)
dataFrame

Data Scientist Jobs


,Job Title,Company,Experience Required,Location
0,Developer - Data Science,ICL Systems India Private Limited,3-5 Yrs,Delhi / NCR
1,Required- Data Scientist (NLP)-Axis Bank - 6 m...,Axis Bank Limited,4-9 Yrs,"Kolkata, New Delhi, Hyderabad/Secunderabad, Pu..."
2,Data Scientist/Data Analyst - Python/Machine L...,Change leaders,5-10 Yrs,"Mumbai, Ghaziabad"
3,Business Consultant Data Scientist,StriveX Consulting Pvt. Ltd,2-4 Yrs,"Hyderabad/Secunderabad, Chennai, Bangalore/Ben..."
4,Data Scientist,Amity University,6-8 Yrs,"Ghaziabad, Faridabad, Delhi / NCR"
5,Data Scientist - Python & Machine Learning,FUTURES AND CAREERS,2-7 Yrs,"Hyderabad/Secunderabad, Pune, Bangalore/Bengal..."
6,Data Scientist - Python & Machine Learning,FUTURES AND CAREERS,2-7 Yrs,"Hyderabad/Secunderabad, Bangalore/Bengaluru, D..."
7,Data Scientist - Machine Learning/ NLP,TalPro,2-6 Yrs,Gurgaon/Gurugram
8,Hiring For Data Scientist,Max Bupa Health Insurance Company Limited,1-6 Yrs,"Gurgaon/Gurugram, Delhi / NCR"
9,Data Scientist - Python / Machine Learning / T...,FUTURES AND CAREERS,3-8 Yrs,"Hyderabad/Secunderabad, Bangalore/Bengaluru, D..."


## Q4: Write a python program to scrape data for first 10 job results for Data scientist Designation in Noida location. You have to scrape company_name, No. of days ago when job was posted, Rating of the company.


This task will be done in following steps:
1. first get the webpage https://www.glassdoor.co.in/index.htm
2. Enter “Data Scientist” in “Job Title,Keyword,Company” field and enter “Noida” in “location” field.
3. Then click the search button. You will land up in the below page:
4. Then scrape the data for the first 10 jobs results you get in the above shown page.
5. Finally create a dataframe of the scraped data.

Note- All of the above steps have to be done in code. No step is to be done
manually.

In [69]:
driver, dataframe = get_highlevel_details_glassdoor('Data Scientist','Noida',stop=10)
driver.quit()
dataframe=dataframe[['Company', 'Job Post Age', 'Ratings']]
dataframe

Data scientist Jobs in Noida


,Company,Job Post Age,Ratings
0,Innovaccer Analytics Private Limited,8d,3.5
1,Asquero,24h,5.0
2,Unyscape Infocom Pvt. Ltd,25d,4.1
3,Biz2Credit Inc,30d+,3.8
4,Techlive,,-
5,Salasar New Age Technologies,30d+,-
6,Adobe,5d,4.4
7,SysQuo Innovation Private Limited,9d,3.0
8,Salasar New Age Technologies,30d+,-
9,IBM,5d,4.0


## Q5: Write a python program to scrape the salary data for Data Scientist designation in Noida location. You have to scrape Company name, Number of salaries, Average salary, Min salary, Max Salary.


The above task will be, done as shown in the below steps:
1. first get the webpage https://www.glassdoor.co.in/Salaries/index.htm
2. Enter “Data Scientist” in Job title field and “Noida” in location field.
3. Click the search button.
4. After that you will land on the below page You have to scrape whole data from this webpage
5. Scrape data for first 10 companies. Scrape the min salary, max salary, company name, Average salary and rating of the company.
6. Store the data in a dataframe.

Note that all of the above steps have to be done by coding only and not manually.

In [74]:
x = ['one','two']
x.extend('three')
print(x)

['one', 'two', 't', 'h', 'r', 'e', 'e']


In [72]:
def login_glassdoor_salary(driver):
    username = 'testacount573@gmail.com'
    paswd = 'Testaccount123'
    
    main_window = driver.current_window_handle
    try:
        driver.find_element_by_xpath("//li[@class='sign-in']/a").click()
    except NoSuchElementException:
        #driver.find_element_by_class_xpath("//div[@class='d-flex']//button").click()
        pass
        
    time.sleep(3)
    #all_windows = driver.window_handles
    #print(all_windows)
    #driver.switch_to.window(all_windows[1])
    
    driver.find_element_by_id("userEmail").send_keys(username)
    driver.find_element_by_id("userPassword").send_keys(paswd)
    driver.find_element_by_xpath("//button[contains(text(),'Sign In')]").click()
    time.sleep(3)
    driver.switch_to.window(main_window)
    return driver

In [ ]:
def get_salary_details_glassdoor(job_title, location, stop=None):
    url = 'https://www.glassdoor.co.in/Salaries/index.htm'
    drivr = get_drive_launch(url)
    drivr = login_glassdoor_salary(drivr)
    
    time.sleep(2)
    
    drivr.find_element_by_id("KeywordSearch").send_keys(job_title)
    drivr.find_element_by_id("LocationSearch").send_keys(Keys.CONTROL + "a")
    drivr.find_element_by_id("LocationSearch").send_keys(Keys.DELETE)
    drivr.find_element_by_id("LocationSearch").send_keys(location)
    drivr.find_element_by_id("HeroSearchButton").click()
    
    time.sleep(5)
    
    

## Q6 : Scrape data of first 100 sunglasses listings on flipkart.com. You have to scrape four attributes:
1. Brand
2. Product Description
3. Price
4. Discount %
The attributes which you have to scrape is ticked marked in the below image.


To scrape the data you have to go through following steps:
1. Go to flipkart webpage by url https://www.flipkart.com/
2. Enter “sunglasses” in the search field where “search for products, brands and more” is written and click the search icon
3. after that you will reach to a webpage having a lot of sunglasses. From this page you can scrap the required data as usual.
4. after scraping data from the first page, go to the “Next” Button at the bottom of the page , then click on it
5. Now scrape data from this page as usual
6. repeat this until you get data for 100 sunglasses.

Note that all of the above steps have to be done by coding only and not manually.

In [49]:
d, dataFrame = get_product_highlevel_details_flipkart('sunglasses', 100)
d.quit()
dataFrame

,Product Brand,Product Description,Product Price,Product Discount
0,PIRASO,UV Protection Round Sunglasses (55),₹449,82% off
1,PIRASO,UV Protection Butterfly Sunglasses (55),₹209,86% off
2,Fastrack,UV Protection Wayfarer Sunglasses (Free Size),₹758,15% off
3,Fastrack,UV Protection Rectangular Sunglasses (Free Size),₹569,28% off
4,Fastrack,"Mirrored, UV Protection Wayfarer Sunglasses (F...",₹499,50% off
...,...,...,...,...
95,ALEYBEE,UV Protection Round Sunglasses (54),₹189,87% off
96,GANSTA,"UV Protection, Night Vision, Riding Glasses Av...",₹295,85% off
97,ROZZETTA CRAFT,"UV Protection, Gradient Round Sunglasses (Free...",₹399,80% off
98,hipe,UV Protection Round Sunglasses (Free Size),₹228,82% off


## Q7: Scrape 100 reviews data from flipkart.com for iphone11 phone. You have to go the link: https://www.flipkart.com/apple-iphone-11-black-64-gb-includesearpods-poweradapter/p/itm0f37c2240b217?pid=MOBFKCTSVZAXUHGR&lid=LSTMOBFKCTSVZAXUHGREPBFGI&marketplace. 


When you will open the above link you will reach to the below shown webpage.

As shown in the above page you have to scrape the tick marked attributes.
These are 
1. Rating 
2. Review_summary 
3. Full review

You have to scrape this data for first 100 reviews.

In [9]:
def get_reviews_flipkart(product_url, no_of_reviews):
    drivr = get_drive_launch(product_url)
    product_reviews_url = drivr.find_element_by_xpath("//div[@class='_3UAT2v _16PBlm']/..").get_attribute('href')
    drivr.quit()
    drivr = get_drive_launch(product_reviews_url)

    time.sleep(2)

    ratings = []
    review_summaries = []
    full_reviews = []

    while len(ratings) < no_of_reviews:
        rating_elems = drivr.find_elements_by_xpath("//div[@class='_3LWZlK _1BLPMq']")
        review_summary_elems = drivr.find_elements_by_xpath("//p[@class='_2-N8zT']")
        full_review_elems= drivr.find_elements_by_xpath("//div[@class='t-ZTKy']")

        for rating_elem,review_summary_elem,full_review_elem in zip(rating_elems,review_summary_elems,full_review_elems):
            ratings.append(rating_elem.text.strip())
            review_summaries.append(review_summary_elem.text.strip())
            full_reviews.append(full_review_elem.text.strip())
            if len(ratings) == no_of_reviews:
                break
        drivr.find_element_by_xpath("//span[contains(text(),'Next')]").click()
        time.sleep(3)
    dataframe = pd.DataFrame({'Rating': ratings, 'Review Summary' : review_summaries, 'Full Review' : full_reviews})
    return drivr, dataframe

In [10]:
product_url = 'https://www.flipkart.com/apple-iphone-11-black-64-gb-includes%02earpods-power%02adapter/p/itm0f37c2240b217?pid=MOBFKCTSVZAXUHGR&lid=LSTMOBFKCTSVZAXUHGREPBFGI&marketplace'
d, DataFrame = get_reviews_flipkart(product_url, 100)
d.quit()
DataFrame

,Rating,Review Summary,Full Review
0,5,Brilliant,The Best Phone for the Money\n\nThe iPhone 11 ...
1,5,Perfect product!,Amazing phone with great cameras and better ba...
2,5,Great product,Amazing Powerful and Durable Gadget.\n\nI’m am...
3,5,Worth every penny,Previously I was using one plus 3t it was a gr...
4,5,Highly recommended,iphone 11 is a very good phone to buy only if ...
...,...,...,...
95,4,Very Good,impressive super phone and best in class camer...
96,5,Perfect product!,Best and amazing product.....phone looks so pr...
97,5,Fabulous!,Everything is perfect pictures come out so cle...
98,5,Terrific,I got this beast today. And I must say the pic...


## 8: Scrape data for first 100 sneakers you find when you visit flipkart.com and search for “sneakers” in the search field.

You have to scrape 4 attributes of each sneaker :
1. Brand
2. Product Description
3. Price
4. discount %

As shown in the below image, you have to scrape the tick marked attributes

Also note that all the steps required during scraping should be done through code 
only and not manually

In [11]:
d, dataFrame = get_product_highlevel_details_flipkart('sneakers', 100)
d.quit()
dataFrame

,Product Brand,Product Description,Product Price,Product Discount
0,K' Footlance,Sneakers For Men,₹499,50% off
1,BRUTON,Combo Pack Of 4 Casual Sneakers For Men,₹449,88% off
2,Chevit,Perfect & Affordable Combo Pack of 02 Pairs Sn...,₹499,72% off
3,Chevit,Combo Pack of 4 Casual Sneakers With Sneakers ...,₹474,76% off
4,Robbie jones,Casual Sneakers Shoes For Men Sneakers For Men,₹360,63% off
...,...,...,...,...
95,SHOEFLY,Combo Men Pack of 2 Loafers Shoes Sneakers For...,₹343,65% off
96,Chevit,Smart Casuals Canvas Shoes Combo pack of 2 Sne...,₹399,60% off
97,BERSACHE,"Combo pack of 2 casual, sneaker and loafer sho...",₹498,50% off
98,World Wear Footwear,Latest Collection-1227 Stylish Casual Sports S...,₹259,48% off


## Q9: Go to the link - https://www.myntra.com/shoes Set Price filter to “Rs. 6649 to Rs. 13099” , Color filter to “Black” and then scrape First 100 shoes data you get. The data should include “Brand” of the shoes , Short Shoe description, price of the shoe Please note that applying the filter and scraping the data , everything should be done through code only and there should not be any manual step.

In [103]:
def get_product_details_myntra(url, num_of_items = 10, use_filter = False, price_range_item_num = 1, color = 'White'):
    drivr = get_drive_launch(url)
    if use_filter:
        drivr.find_element_by_xpath("//ul[@class='price-list']/li["+str(price_range_item_num)+"]").click()
        time.sleep(2)
        try:
            drivr.find_element_by_xpath("//li[@class='colour-listItem']/label[(contains(text(),'"+color+"'))]").click()
        except:
            drivr.find_element_by_class_name("colour-more").click()
            drivr.find_element_by_xpath("//li[@class='colour-listItem']/label[(contains(text(),'"+color+"'))]").click()
        time.sleep(2)

    product_names = []
    product_short_desc = []
    prices = []

    while (len(product_names) < num_of_items):        
        brand_name_elems = drivr.find_elements_by_xpath("//h3[@class='product-brand']")
        short_desc_elems = drivr.find_elements_by_xpath("//h4[@class='product-product']")        
        price_elems = drivr.find_elements_by_xpath("//div[@class='product-price']")
        for brand_name, short_desc, price in zip(brand_name_elems, short_desc_elems, price_elems):
            product_names.append(brand_name.text)
            product_short_desc.append(short_desc.text)
            prices.append(price.text.split('Rs. ')[1])
            if len(product_names) == num_of_items:
                break
        if len(product_names) == num_of_items:
                break
        url = drivr.find_element_by_xpath("//a[@rel='next']").get_attribute('href')
        drivr.quit()
        drivr = get_drive_launch(url)
        time.sleep(3)
    datafrm = pd.DataFrame({'Product Name':product_names, 'Product Short Description': product_short_desc, 'Price in INR':prices})
    return drivr, datafrm


In [105]:
url = 'https://www.myntra.com/shoes'
d, df = get_product_details_myntra(url, 100, True, 2, 'Black')
d.quit()
df


,Product Name,Product Short Description,Price in INR
0,Puma,Men Fuse Training Sports Shoes,7999
1,Nike,AIR ZOOM PEGASUS Running Shoes,11495
2,Nike,Men AIR ZOOM Running Shoes,11470
3,PUMA Motorsport,Unisex Mercedes Running Shoes,7999
4,Nike,Men REACT MILER Running Shoes,9345
...,...,...,...
95,VIONIC,Men Textured Sneakers,7039
96,Hush Puppies,Men Colourblocked Driving Shoes,8999
97,Puma,Men IGNITE Dual Running Shoes,6999
98,Puma,Men IGNITE Ronin Unrest,6999


In [ ]:
def get_product_details_myntra(url, num_of_items = 10, use_filter = False, price_range_item_num = 1, color = 'White'):
    drivr = get_drive_launch(url)
    if use_filter:
        drivr.find_element_by_xpath("//ul[@class='price-list']/li["+str(price_range_item_num)+"]").click()
        time.sleep(2)
        try:
            drivr.find_element_by_xpath("//li[@class='colour-listItem']/label[(contains(text(),'"+color+"'))]").click()
        except:
            drivr.find_element_by_class_name("colour-more").click()
            drivr.find_element_by_xpath("//li[@class='colour-listItem']/label[(contains(text(),'"+color+"'))]").click()
        time.sleep(2)

    product_names = []
    product_short_desc = []
    prices = []

    while (len(product_names) < num_of_items):        
        brand_name_elems = drivr.find_elements_by_xpath("//h3[@class='product-brand']")
        short_desc_elems = drivr.find_elements_by_xpath("//h4[@class='product-product']")        
        price_elems = drivr.find_elements_by_xpath("//div[@class='product-price']")
        for brand_name, short_desc, price in zip(brand_name_elems, short_desc_elems, price_elems):
            product_names.append(brand_name.text)
            product_short_desc.append(short_desc.text)
            prices.append(price.text.split('Rs. ')[1])
            if len(product_names) == num_of_items:
                break
        if len(product_names) == num_of_items:
                break
        url = drivr.find_element_by_xpath("//a[@rel='next']").get_attribute('href')
        drivr.quit()
        drivr = get_drive_launch(url)
        time.sleep(3)
    datafrm = pd.DataFrame({'Product Name':product_names, 'Product Short Description': product_short_desc, 'Price in INR':prices})
    return drivr, datafrm


## Q10: Go to webpage https://www.amazon.in/ Enter “Laptop” in the search field and then click the search icon. Then set CPU Type filter to “Intel Core i7” and “Intel Core i9” After setting the filters scrape first 10 laptops data. 


You have to scrape 3 attributes for each laptop:
1. title
2. Ratings
3. Price

In [111]:
url = 'https://www.amazon.in/'
drivr = get_drive_launch(url)
drivr.find_element_by_id("twotabsearchtextbox").send_keys("laptop")
drivr.find_element_by_id("nav-search-submit-text").click()
time.sleep(2)
drivr.find_element_by_xpath("//span[contains(text(),'Intel Core i7')]").click()
time.sleep(2)
drivr.find_element_by_xpath("//span[contains(text(),'Intel Core i9')]").click()

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//span[contains(text(),'Intel Core i9')]"}
  (Session info: chrome=89.0.4389.90)


In [ ]:

if use_filter:
    drivr.find_element_by_xpath("//ul[@class='price-list']/li["+str(price_range_item_num)+"]").click()
    time.sleep(2)
    try:
        drivr.find_element_by_xpath("//li[@class='colour-listItem']/label[(contains(text(),'"+color+"'))]").click()
    except:
        drivr.find_element_by_class_name("colour-more").click()
        drivr.find_element_by_xpath("//li[@class='colour-listItem']/label[(contains(text(),'"+color+"'))]").click()
    time.sleep(2)

product_names = []
product_short_desc = []
prices = []

while (len(product_names) < num_of_items):        
    brand_name_elems = drivr.find_elements_by_xpath("//h3[@class='product-brand']")
    short_desc_elems = drivr.find_elements_by_xpath("//h4[@class='product-product']")        
    price_elems = drivr.find_elements_by_xpath("//div[@class='product-price']")
    for brand_name, short_desc, price in zip(brand_name_elems, short_desc_elems, price_elems):
        product_names.append(brand_name.text)
        product_short_desc.append(short_desc.text)
        prices.append(price.text.split('Rs. ')[1])
        if len(product_names) == num_of_items:
            break
    if len(product_names) == num_of_items:
            break
    url = drivr.find_element_by_xpath("//a[@rel='next']").get_attribute('href')
    drivr.quit()
    drivr = get_drive_launch(url)
    time.sleep(3)
pd.DataFrame({'Product Name':product_names, 'Product Short Description': product_short_desc, 'Price in INR':prices})

